# Dataset for the Arithmetic Equation Simplification (AES) Task

In [1]:
# dependency
# public
import os
import numpy as np
from tqdm import tqdm
# private
from utils import save_txt

In [2]:
# class for data generation of the Arithmetic Equation Simplification (AES) problem 
class ArithmeticEquationSimplification(): 
    """docstring for ArithmeticEquationSimplification"""
    def __init__(self, operators, num_size):
        super().__init__()
        self.operators = operators
        self.pos_digits = np.arange(2, num_size+2).tolist()
        self.neg_digits = np.arange(-num_size, -1).tolist()
        self.digits = self.pos_digits + self.neg_digits
    
    def gen_base_dict(self):
        return {str(i):[] for i in self.pos_digits}
    
    def expand_base_dict(self):
        for a in self.digits:
            for o in self.operators:
                for b in self.pos_digits:
                    try:
                        e = [str(a), o, str(b)]
                        v = str(eval(''.join(e)))
                        e[0] = e[0].replace('-', '- ')
                        e = ' '.join(list(e))
                        if v in self.base_dict:
                            self.base_dict[v].append('( {} )'.format(e))
                    except:
                        pass
    
    def gen_operation(self, seq_len):
        if seq_len == 1:
            a = np.random.choice(self.digits)
            return [str(a)]
        else:
            left_side  = self.gen_operation(seq_len-1)
            o = np.random.choice(self.operators)
            b = np.random.choice(self.pos_digits)
            return left_side + [o, str(b)]
    
    def gen_operation_list(self, seq_len, data_size):
        # to control the data size
        operations_pool = set()
        for i in tqdm(range(data_size)):
            while True: 
                # to avoid duplicates
                operation = self.gen_operation(seq_len) 
                if ''.join(operation) in operations_pool: 
                    continue
                else:
                    operations_pool.add(''.join(operation)) 
                # to avoid zero division error
                try: 
                    # flost to int to string
                    value = eval(''.join(operation))
                    if value % 1 != 0.: 
                        continue
                    else:
                        value = str(int(value))
                        # to keep vocab size
                        if value in self.value_dict: 
                            self.value_dict[value].append(operation)
                            break
                except: 
                    pass
    
    def gen_equation_list(self):
        ys = []
        for v in self.value_dict:
            for y in self.value_dict[v]:
                y = y[0].replace('-', '- ').split() + y[1:]
                y += ["=="] + [v]
                ys.append(' '.join(y))
        return ys
    
    def replace_numbers(self, ys):
        xs = []
        for y in ys:
            y = y.split()
            num_idx = [i for i, token in enumerate(y) if token.isdigit()] 
            num_to_replace = np.random.choice(range(len(num_idx)+1))
            idx_to_replace = np.random.choice(num_idx, num_to_replace, False)
            for i in idx_to_replace:
                y[i] = np.random.choice(self.base_dict[y[i]])
            xs.append(' '.join(y))
        return xs
                
    def generate(self, seq_len, data_size):
        # input sequences, output sequences
        xs, ys = [], []
        self.base_dict = self.gen_base_dict()
        self.value_dict = self.gen_base_dict()
        self.expand_base_dict()
        self.gen_operation_list(
            seq_len=seq_len, 
            data_size=data_size)
        ys = self.gen_equation_list()
        xs = self.replace_numbers(ys)
        
        return xs, ys

In [10]:
# data parameters 
seq_len = 5
num_size = 100
data_size = 100
operators = ['+', '-', '*', '/']

In [11]:
aes = ArithmeticEquationSimplification(operators, num_size)
xs, ys = aes.generate(seq_len-1, data_size)

100%|██████████| 100/100 [00:00<00:00, 428.61it/s]


In [12]:
sum([x == y for x, y in zip(xs, ys)])/len(xs)

0.24

In [13]:
len(xs)

100

In [14]:
xs[:15]

['( - 8 + 48 ) - ( 58 - 28 ) - 77 + ( 55 + 14 ) == ( - 85 + 87 )',
 '- 40 + ( 72 + 6 ) - ( 92 - 40 ) + 17 == 3',
 '- 65 + 69 * 85 / ( 28 + 57 ) == ( - 97 + 101 )',
 '42 + 11 - 98 + 49 == ( 78 - 74 )',
 '- ( - 68 + 90 ) - ( 10 + 52 ) + 53 + ( 7 * 5 ) == 4',
 '( 60 - 45 ) / ( 70 + 29 ) * ( 62 - 59 ) * ( - 79 + 90 ) == 5',
 '7 + 35 - 43 + 6 == 5',
 '- 33 + 25 + 75 - 60 == ( 30 - 23 )',
 '( 37 - 32 ) + ( 66 - 49 ) + ( - 71 + 89 ) - ( 7 + 26 ) == ( 29 - 22 )',
 '101 - 81 - 30 + 18 == 8',
 '- ( 22 + 55 ) - ( 93 - 17 ) + 83 + 78 == ( 48 - 40 )',
 '( 88 - 47 ) - ( 15 + 86 ) + 13 + ( - 15 + 73 ) == 11',
 '87 + ( - 26 + 84 ) - ( 46 + 35 ) - 52 == ( 90 - 78 )',
 '- ( 37 - 4 ) + ( - 32 + 99 ) - ( 67 + 14 ) + ( 85 - 25 ) == ( - 80 + 93 )',
 '34 + 19 - 26 - 14 == 13']

In [15]:
len(ys)

100

In [16]:
ys[:15]

['40 - 30 - 77 + 69 == 2',
 '- 40 + 78 - 52 + 17 == 3',
 '- 65 + 69 * 85 / 85 == 4',
 '42 + 11 - 98 + 49 == 4',
 '- 22 - 62 + 53 + 35 == 4',
 '15 / 99 * 3 * 11 == 5',
 '7 + 35 - 43 + 6 == 5',
 '- 33 + 25 + 75 - 60 == 7',
 '5 + 17 + 18 - 33 == 7',
 '101 - 81 - 30 + 18 == 8',
 '- 77 - 76 + 83 + 78 == 8',
 '41 - 101 + 13 + 58 == 11',
 '87 + 58 - 81 - 52 == 12',
 '- 33 + 67 - 81 + 60 == 13',
 '34 + 19 - 26 - 14 == 13']

In [13]:
# train val test split
dataset = np.array([(x, y) for x, y in zip(xs, ys)])
data_size = dataset.shape[0]
indices = np.random.permutation(data_size)
train_size = int(0.7*data_size)
val_size = int(0.15*data_size)
test_size = data_size - train_size - val_size
train_idxes = indices[:train_size]
val_idxes = indices[train_size: train_size+val_size]
test_idxes = indices[train_size+val_size:]
trainset = dataset[train_idxes]
valset = dataset[val_idxes]
testset = dataset[test_idxes]
print('train size', train_size, trainset.shape)
print('val size', val_size, valset.shape)
print('test size', test_size, testset.shape)

train size 7000 (7000, 2)
val size 1500 (1500, 2)
test size 1500 (1500, 2)


In [14]:
# to save dataset
outdir = 'aes/'
outdir = os.path.join(
    outdir, 
    'num_size_{}'.format(num_size), 
    'seq_len_{}'.format(seq_len+1), 
    'data_size_{}'.format(data_size))
if not os.path.exists(outdir): 
    os.makedirs(outdir)
outdir

'aes/num_size_100/seq_len_5/data_size_10000'

In [15]:
save_txt(os.path.join(outdir, 'train_x.txt'), trainset[:, 0])
save_txt(os.path.join(outdir, 'train_y.txt'), trainset[:, 1])
save_txt(os.path.join(outdir, 'val_x.txt'), valset[:, 0])
save_txt(os.path.join(outdir, 'val_y.txt'), valset[:, 1])
save_txt(os.path.join(outdir, 'test_x.txt'), testset[:, 0])
save_txt(os.path.join(outdir, 'test_y.txt'), testset[:, 1])